## Lab 2 
## Ньяти Каелиле БВТ2201

### Question 1: Akinator-like program 

In [4]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import json


class Teacher:
    def __init__(self, name, gender, faculty, department, years_experience, associate_professor, assistant, department_head, head_campus, module):
        self.name = name
        self.gender = gender
        self.faculty = faculty
        self.department = department
        self.years_experience = years_experience
        self.associate_professor = associate_professor
        self.assistant = assistant
        self.department_head = department_head
        self.head_campus = head_campus
        self.module = module

    def to_dict(self):
        return {
            "name": self.name,
            "gender": self.gender,
            "faculty": self.faculty,
            "department": self.department,
            "years_experience": self.years_experience,
            "associate_professor": self.associate_professor,
            "assistant": self.assistant,
            "department_head": self.department_head,
            "head_campus": self.head_campus,
            "module": self.module
        }

    @staticmethod
    def from_dict(data):
        return Teacher(
            name=data["name"],
            gender=data["gender"],
            faculty=data["faculty"],
            department=data["department"],
            years_experience=data["years_experience"],
            associate_professor=data["associate_professor"],
            assistant=data["assistant"],
            department_head=data["department_head"],
            head_campus=data["head_campus"],
            module=data["module"]
        )

# File to store teacher data
DATA_FILE = "teachers.json"

# Load teacher database from JSON file
def load_teachers():
    with open(DATA_FILE, "r", encoding="utf-8") as f:
        data = json.load(f)
        return [Teacher.from_dict(entry) for entry in data["teachers"]]
    
# Save teacher database to JSON file
def save_teachers(teachers):
    with open(DATA_FILE, "w", encoding="utf-8") as f:
        json.dump({"teachers": [teacher.to_dict() for teacher in teachers]}, f, ensure_ascii=False, indent=4)


class ExpertSystemApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Teacher Identification Expert System: MTUCI")

        self.teacher_db = load_teachers()
        self.possible_teachers = self.teacher_db[:]
        self.question_index = 0
        self.user_answers = {}

        # list of questions and attributes
        self.questions = [
            # Question 1
            ("Is the person in question Male or Female?", "gender", ["Male", "Female"]),

            # Question 2
            ("Which faculty does the person in question work under?", "faculty", [
                "Information Technology",
                "Cybernetics and Information Security"
            ]),

            # Question 3
            ("Which department does the person in question belong to?", "department", [
                "Mathematical Cybernetics and IT",
                "Corporate Information Systems",
                "Mathematical Analysis",
                "Probability and Applied Mathematics",
                "Physical Education"
            ]),

            # Question 4
            ("Does the teacher have more than 10 years of experience?", "years_experience", 10),

            # Question 5
            ("Is the person in question an Associate Professor?", "associate_professor", True),

            # Question 6
            ("Is the person in question an Assistant instead?", "assistant", True),

            # Question 7
            ("Is the person in question the head of their department?", "department_head", True),

            # Question 8
            ("In which campus is the person in question located?", "head_campus", [
                "Narodnoe Opolchenie",
                "Aviamatornaya",
                "Both"
            ]),

            # Question 9
            ("Which module is the person in question leading or teaching?", "module", [
                "Information Technology and Programming",
                "Structure and Algorithms for Data Processing",
                "Practical Project",
                "Discrete Mathematics",
                "Higher Mathematics",
                "Mathematical Methods in Big Data",
                "None",
                "Probability and Statistics",
                "Sport",
                "Databases"
            ])
        ]

        self.question_label = tk.Label(self.root, text="", wraplength=400, justify="left")
        self.question_label.pack(pady=20)

        # Yes/No buttons
        self.yes_button = tk.Button(self.root, text="Yes", width=15, command=lambda: self.answer_question(True))
        self.no_button = tk.Button(self.root, text="No", width=15, command=lambda: self.answer_question(False))

        # Multiple option buttons
        self.option_buttons = []

        # Initially pack Yes/No buttons
        self.yes_button.pack(side=tk.LEFT, padx=10)
        self.no_button.pack(side=tk.RIGHT, padx=10)

        self.next_question()

    def answer_question(self, answer):
        question_data = self.questions[self.question_index]
        attribute = question_data[1]
        value = question_data[2]

        self.user_answers[attribute] = answer

        if isinstance(value, int):  # Numerical comparison (e.g., years of experience)
            if answer:
                self.possible_teachers = [t for t in self.possible_teachers if getattr(t, attribute) >= value]
            else:
                self.possible_teachers = [t for t in self.possible_teachers if getattr(t, attribute) < value]
        else:
            if answer in ["Male", "Female"]:
                # This handles the gender question specifically
                self.possible_teachers = [t for t in self.possible_teachers if getattr(t, attribute) == answer]
            elif attribute in ["faculty", "department", "head_campus", "module"]:
                # These are handled via option buttons, so no need to filter here
                pass
            else:
                if answer:
                    self.possible_teachers = [t for t in self.possible_teachers if getattr(t, attribute) == value]
                else:
                    self.possible_teachers = [t for t in self.possible_teachers if getattr(t, attribute) != value]

        self.check_remaining_teachers()

    def answer_option(self, option):
        question_data = self.questions[self.question_index]
        attribute = question_data[1]

        self.user_answers[attribute] = option

        self.possible_teachers = [t for t in self.possible_teachers if getattr(t, attribute) == option]

        self.check_remaining_teachers()

    def check_remaining_teachers(self):
        if len(self.possible_teachers) == 1:
            self.show_result(self.possible_teachers[0].name)
        elif len(self.possible_teachers) == 0:
            self.add_teacher()
        elif self.question_index + 1 < len(self.questions):
            self.question_index += 1
            self.next_question()
        else:
            self.show_result_multiple()

    def next_question(self):
        # Clear any existing option buttons
        for btn in self.option_buttons:
            btn.destroy()
        self.option_buttons = []

        if self.question_index < len(self.questions):
            question_data = self.questions[self.question_index]
            self.question_label.config(text=question_data[0])

            if isinstance(question_data[2], list):
                # Show options for the user to select (e.g., for departments)
                for option in question_data[2]:
                    btn = tk.Button(self.root, text=option, width=30, wraplength=200, justify="left",
                                   command=lambda opt=option: self.answer_option(opt))
                    btn.pack(pady=5)
                    self.option_buttons.append(btn)
                # Hide Yes/No buttons in this case
                self.yes_button.pack_forget()
                self.no_button.pack_forget()
            else:
                # Show Yes/No buttons for binary questions
                self.yes_button.pack(side=tk.LEFT, padx=10)
                self.no_button.pack(side=tk.RIGHT, padx=10)

    def show_result(self, teacher_name):
        messagebox.showinfo("Result", f"The teacher you're thinking of is:\n\n{teacher_name}")
        self.root.quit()

    def show_result_multiple(self):
        if len(self.possible_teachers) > 1:
            possible_names = [t.name for t in self.possible_teachers]
            names_str = "\n".join(possible_names)
            messagebox.showinfo("Result", f"Possible teachers are:\n\n{names_str}")
        else:
            messagebox.showinfo("Result", "No matching teacher found.")
            self.add_teacher()
        self.root.quit()

    def add_teacher(self):
        add_teacher = messagebox.askyesno("Add Teacher", "No matching teacher found. Would you like to add this teacher?")
        if add_teacher:
            name = simpledialog.askstring("Input", "Enter the teacher's name:")
            if not name:
                messagebox.showerror("Error", "Name cannot be empty.")
                return

            # Ask for years of experience and convert to integer
            years_experience_str = simpledialog.askstring("Input", "Enter the number of years of experience:")
            if years_experience_str is None or not years_experience_str.isdigit():
                messagebox.showerror("Error", "Please enter a valid number for years of experience.")
                return
            years_experience = int(years_experience_str)

            gender = self.user_answers.get("gender", "Unknown")
            faculty = self.user_answers.get("faculty", "Unknown")
            department = self.user_answers.get("department", "Unknown")
            associate_professor = self.user_answers.get("associate_professor", False)
            assistant = self.user_answers.get("assistant", False)
            department_head = self.user_answers.get("department_head", False)
            head_campus = self.user_answers.get("head_campus", "Unknown")
            module = self.user_answers.get("module", "None")

            # Handle cases where the answer might not have been set
            if isinstance(gender, bool):
                gender = "Male" if gender else "Female"  
            if isinstance(head_campus, bool):
                head_campus = "Aviamatornaya" if head_campus else "Narodnoe Opolchenie"
             
            new_teacher = Teacher(
                name=name,
                gender=gender,
                faculty=faculty,
                department=department,
                years_experience=years_experience,
                associate_professor=associate_professor,
                assistant=assistant,
                department_head=department_head,
                head_campus=head_campus,
                module=module
            )

            self.teacher_db.append(new_teacher)
            save_teachers(self.teacher_db)
            messagebox.showinfo("Teacher Added", f"Teacher '{name}' has been added to the database.")

        self.root.quit()

# Create the main window
def main():
    root = tk.Tk()
    app = ExpertSystemApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()
